In [1]:
# Copyright 2023, Acadential, All rights reserved.

# 9-5. Save and Load Model

학습된 모델을 저장하고 저장된 모델을 불러오는 과정을 살펴보겠습니다.

Terminology:
- checkpoint: 일반적으로 모델의 parameter을 저장한 파일을 의미. 

~~~
# To save model
torch.save(model, 'model.pth')

# To load model
model = torch.load('model.pth')
~~~

모델을 저장하고 불러오는 방법은 2가지 방법이 있습니다:
1. 모델을 checkpoint에 **"통째로"** 저장하고 불러오는 방법.
2. **모델의 parameter와 기타 다른 정보들 (test accuracy, 등등)** 을 checkpoint에 함께 저장하고 불러오는 방법


In [1]:
import torch
from torch import nn
from src.model import NeuralNetwork

In [2]:
model = NeuralNetwork()

In [3]:
model

NeuralNetwork(
  (fc_layers): Sequential(
    (0): Linear(in_features=784, out_features=196, bias=True)
    (1): ReLU()
    (2): Linear(in_features=196, out_features=49, bias=True)
    (3): ReLU()
    (4): Linear(in_features=49, out_features=10, bias=True)
    (5): Sigmoid()
  )
)

In [4]:
import os 
# checkpoints 폴더 생성 후 모델 저장
os.makedirs("checkpoints", exist_ok=True)
checkpoint_path = "checkpoints/sample_model.pth"


# 1. 모델을 통째로 저장하는 방법

이 방법으로 모델을 load하려고 했을시 Neural Network model에 대한 python 코드가 없어도 괜찮습니다.

## Save

In [5]:
# save model
torch.save(model, checkpoint_path)

## Load

In [6]:
loaded_model = torch.load(checkpoint_path)

In [7]:
loaded_model

NeuralNetwork(
  (fc_layers): Sequential(
    (0): Linear(in_features=784, out_features=196, bias=True)
    (1): ReLU()
    (2): Linear(in_features=196, out_features=49, bias=True)
    (3): ReLU()
    (4): Linear(in_features=49, out_features=10, bias=True)
    (5): Sigmoid()
  )
)

# 2. 모델의 parameter와 기타 정보들을 함께 저장하는 방법

이 방법으로 모델을 load하려면 Neural Network model을 먼저 initiate해야합니다. (즉, Neural Network model에 대한 python 코드가 필요함)

## Save

In [8]:
checkpoint_path = "checkpoints/sample_checkpoint.pt"
# 딕셔너리 형태로 명시
content = {
    "model_state_dict": model.state_dict(),
    "epochs": 100,
    "test_accuracy": 0.9,
    "lr": 0.001,
}

torch.save(content, checkpoint_path)


## Load

In [9]:
loaded_checkpoint = torch.load(checkpoint_path)
loaded_checkpoint.keys()

dict_keys(['model_state_dict', 'epochs', 'test_accuracy', 'lr'])

In [12]:
for key in loaded_checkpoint.keys():
    if key != "model_state_dict":
        print(f"{key} =", loaded_checkpoint[key])

epochs = 100
test_accuracy = 0.9
lr = 0.001


### Initiate NN model

In [13]:
model2 = NeuralNetwork()

In [14]:
model2.load_state_dict(loaded_checkpoint["model_state_dict"])

<All keys matched successfully>

# Using pre-trained models from torchvision

PyTorch의 torchvision에서는 다양한 Machine Vision Neural Network 모델들과 model checkpoint들을 제공합니다. \
대표적으로 제공되는 모델들:

1. ResNet
2. DenseNet
3. MobileNet V2, V3
4. Swin Transformer
5. etc.



In [15]:
from torchvision.models import resnet50

In [16]:
model = resnet50(pretrained=False) # use randomly initiated weight

e:\Study\Python\DL\venvDL\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\Study\Python\DL\venvDL\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [17]:
model = resnet50(pretrained=True)  # use ImageNet pretrained weight

e:\Study\Python\DL\venvDL\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\82104/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 56.9MB/s]


In [18]:
out = model(torch.rand(1, 3, 64, 64))

In [19]:
out.shape

torch.Size([1, 1000])

# Advanced loading

만약에 pretrained된 모델의 weight을 사용하고 싶은데 마지막 classification layer만 randomly initialize하고 싶을때는 어떻게 할까요?

예를 들어 ImageNet의 output class 개수는 1000개이지만 CIFAR 10의 경우 output class 개수는 10개입니다.

그럴 경우 Last layer을 제외한 나머지 layer들의 weight들만 pretrained model checkpoint의 weight으로 initialize해줍니다! 

In [20]:
from torchvision.models.resnet import ResNet50_Weights
from torch.hub import load_state_dict_from_url

In [21]:
checkpoint = load_state_dict_from_url(ResNet50_Weights.IMAGENET1K_V2.url)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\82104/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 56.7MB/s]


In [22]:
model = resnet50(pretrained=False, num_classes=10)

## Size Mismatch Error

다음과 같이 output class 개수가 다르면 마지막 classification layer의 weight (matrix of shape (Hidden x Number of class) )가 다르기 때문에 에러가 뜹니다.

In [23]:
model.load_state_dict(checkpoint, strict=False)

RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([1000, 2048]) from checkpoint, the shape in current model is torch.Size([10, 2048]).
	size mismatch for fc.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([10]).

## 마지막 Layer의 weight만 제외해서 checkpoint 불러오기

In [28]:
from collections import OrderedDict

layers_to_filter_out = []
model_state_dict = model.state_dict()

# checkpoint의 레이어들을 순회하면서 각 레이어의 weight의 크기가 다르다면
# FilterOut 대상이 되도록 해당 레이어는 LayersToFilterOut 리스트에 append
# 해주도록 하겠습니다.
for layer in checkpoint.keys():
    if model_state_dict[layer].shape != checkpoint[layer].shape:
        layers_to_filter_out.append(layer)
# 어떤 레이어들이 filter out대상이 되었는지 확인하기 위해 프린트문을 작성해본다.

print("Layers to filter out", layers_to_filter_out)
for layer in layers_to_filter_out:
    del checkpoint[layer]

model.load_state_dict(checkpoint, strict=False)


Layers to filter out ['fc.weight', 'fc.bias']


_IncompatibleKeys(missing_keys=['fc.weight', 'fc.bias'], unexpected_keys=[])

In [26]:
checkpoint.keys()

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.we

In [27]:
checkpoint['conv1.weight'].shape

torch.Size([64, 3, 7, 7])